# Covid Analysis

This notebook analyzes the US Covid data from this web site: [The Covid Tracker: Data](https://covidtracking.com/data/download).  the data definitions can be found on a page linked form that initial page: [The Covid Tracker: Data Definition](https://covidtracking.com/about-data/data-definitions).

The first level of analysis is to performa a moving average to find the 'average' shape of the curve for number of new cases, which is indicated by the field name <font face='courier'>positiveIncrease</font>.

This example is for illustration of a data analysis and so, to save a bit of effort we will not worry about formatting the x-axis tickmark labels for clarity.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

## Data Input with <font face='courier'>pandas</font>

In [ ]:
df = pd.read_csv('national-history.csv')
df.head()

In [ ]:
df_nc = df[['date', 'positiveIncrease']]
df_nc.head()

In [ ]:
df_nc.dtypes

Let's re-index the data so that it is from earliest to latest date chronologially, and then look at it.

In [ ]:
df_nc = df_nc.reindex(index=df_nc.index[::-1])
df_nc.reset_index(inplace=True, drop=True)
df_nc.head()

Visualize the data.  What patterns or characteristics do you see?

In [ ]:
fig,ax = plt.subplots()
ax.plot(df_nc.index, df_nc.positiveIncrease, c='k')
ax.set_xlabel('Day Number')
ax.set_ylabel('Number of New Cases')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
#ax.vlines(100, 0, 75000, linestyles='dashed', colors=['gray'])
plt.show()

Next:

- Let's compute and view a moving average to determine the overall average trend.
- Then, subtract the trend from the original data to compute and view the remainder.  Are there obvious patterns in the remainder?

The <font face='courier'>pandas</font> <font face='courier'>.rolling()</font> method picks out a series of rolling windows of specified length, jsut as we need for a moving average.  Applying <font face='courier'>.mean()</font> computes the average of each successive window.

The idea here is to try multiple windows lengths to find a window length that gives a smooth appearance and looks to be a reasonable indication of the aaverage trend of the data.  None of the variability of the original data (e.g., seasonality and short-term cyclicality sould be reflected in this representation of the overall trend.  Notice that applying a moving average causes a decreased number of data points as is represented by the <font face='courier'>NaN</font> (not a number, null) values.

In [ ]:
mv_avg = df_nc.positiveIncrease.rolling(20, center=False).mean()
mv_avg.head(n=25)

Compute moving averages with <font face='courier'>pandas</font> <font face='courier'>.rolling()</font> method, insert it into the <font face='courier'>DataFrame</font> and, subsequently, compute the first residual.  Note that the argument <font face='courier'>center</font> is now set to <font face='courier'>True</font>: we will discuss its significance.  Note the <font face='courier'>NaN</font> values.

In [ ]:
mv_avg_period = 15
df_nc['mv_avg'] = df_nc.positiveIncrease.rolling(mv_avg_period, center = True).mean()
df_nc['R1'] = df_nc.positiveIncrease - df_nc.mv_avg
df_nc[['mv_avg', 'R1']].iloc[mv_avg_period - 10 : mv_avg_period + 10]

Compute sum of squared errors of current residual as an indication of fit.

In [ ]:
print('Sum of Squared Errors: %e' % (df_nc.R1**2).sum())

In [ ]:
fig,ax = plt.subplots(2,1, sharex=True)
ax[0].plot(df_nc.index,df_nc.positiveIncrease, c='k', label='Data')
ax[0].plot(df_nc.index, df_nc.mv_avg, c='r', linestyle='--', label='Moving Avg.')
ax[0].set_xlabel('Day Number')
ax[0].set_ylabel('Number of New Cases')
#ax[0].vlines(100, ax[0].get_ylim()[0],  ax[0].get_ylim()[1], linestyles='dashed', colors=['gray'])
#ax[0].vlines(190, ax[0].get_ylim()[0],  ax[0].get_ylim()[1], linestyles='dashed', colors=['gray'])
ax[0].legend()
ax[0].set_title('Moving Average')
ax[1].plot(df_nc.index, df_nc.R1, c='k')
ax[1].set_xlabel('Day Number')
ax[1].set_ylabel('Remainder After Moving Average')
ax[1].set_title('Remainder')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,11)
plt.show()

# A Linear Trend Model

In [ ]:
from scipy import stats

slope, intercept, r_value, p_value, std_err = stats.linregress(df_nc.index, df_nc.positiveIncrease)
print('intercept =', intercept, '    slope =', slope, '     p_value = ',p_value)

In [ ]:
df_nc['regress'] = intercept + slope * df_nc.index
df_nc['R1_lin_trend'] = df_nc.positiveIncrease - df_nc.regress
print('Sum of Squared Errors: %e' % (df_nc.R1_lin_trend**2).sum())

Plot the data and the linear fit.  Note visually the remainder.

In [ ]:
fig,ax = plt.subplots(2,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
ax[0].plot(df_nc.index,df_nc.regress, c='r', linestyle='--', label='Regression Model')
ax[0].set_xlabel('Day Number')
ax[0].set_ylabel('Number of New Cases')
#ax[0].vlines(100, ax[0].get_ylim()[0],  ax[0].get_ylim()[1], linestyles='dashed', colors=['gray'])
#ax[0].vlines(190, ax[0].get_ylim()[0],  ax[0].get_ylim()[1], linestyles='dashed', colors=['gray'])
ax[0].legend()
ax[1].plot(df_nc.index, df_nc.R1_lin_trend,c='k')
ax[1].set_xlabel('Day Number')
ax[1].set_ylabel('Remainder After Moving Average')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,10)
plt.show()

## Autocorrelation Analysis (with nonlinear model)

The results below demonstrate a clear pattern of 7-day cyclicality.  What might be causing it?

Note that a period of 7 days is consistent with a weekly repeating cyclicality.  One issue with computing this cyclicality is that the magnitude of the current remainder is not of a constant magnitude.

In [ ]:
for i in range(1,21):
    print('Correlation with lag %d:  %7.5f' % (i,df_nc['R1'].autocorr(lag = i)))

The cell below computes the average value of the <font face='courier'>R1</font> values for each day of the week (e.g., Sunday, Monday, etc.) over the data range because a lag of 7 was positive and of the greatest magnitude, as shown in the cell above.  Note that adjustments to the data indices is required, using <font face='courier'>int(mv_avg_period/2)</font> to avoid making computations with the <font face='courier'>R1</font> values of <font face='courier'>NaN</font>.

In [ ]:
lag = 7
num_rows = df_nc.shape[0] - mv_avg_period
cycl = [sum([df_nc.R1.iloc[int(mv_avg_period/2) + j*lag + i] for j in range(int(num_rows/lag))])/int(num_rows/lag) for i in range(lag)]
print(sum(cycl)*int(num_rows/lag), sum(df_nc.R1[int(mv_avg_period/2):int(df_nc.shape[0] -(mv_avg_period/2))]))
print(cycl)

In [ ]:
df_nc[['R1','mv_avg']].iloc[:20]

A <font face='courier'>DataFrame</font> column for the cyclicality pattern is created in the  next cell, and also a column for the second residual, <font face='courier'>R2</font>.  The fit so far is assesed by computing the Sum of Squared Errors for <font face='courier'>R2</font>.

Note that the first cyclic average may not correspond to the first row in the <font face='courier'>DataFrame</font>.  You must ensure that the first cyclic average computed is associated with the first <font face='courier'>DataFrame</font> row that does not have <font face='courier'>NaN</font> values in the moving average and <font face='courier'>R1</font> columns: this row's <font face='courier'>R1</font> value corresponds with the first cyclic average.  To do that, you can adjust the "offset" represented by the variable <font face='courier'>offset</font> in the statement below.  The appropriate value of <font face='courier'>offset</font> depends on the moving average window length you used.  The variable <font face='courier'>lag</font> refers to the period of the cyclicality, which is the lag of the greatest positive autocorrelation.

In [ ]:
# The offset variable compensate for the first seasonal factor being for period 10, which shuld refer to season 0
offset = 7
df_nc['cycl'] = [cycl[(i - offset)%lag] for i in range(df_nc.shape[0])]
df_nc['R2'] = df_nc.R1 - df_nc.cycl
df_nc.cycl.iloc[:20]

Note the reduction in Sum of Squared Errors

In [ ]:
print('Sum of Squared Errors: %e' % (df_nc.R2**2).sum())

In [ ]:
fig,ax = plt.subplots(3,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
ax[0].plot(df_nc.index,df_nc.mv_avg, c='r', linestyle='--', label='Moving Avg.')
ax[0].set_xlabel('Day Number')
ax[0].set_ylabel('Number of New Cases')
ax[0].legend()
ax[0].set_title('Moving Average')
ax[1].plot(df_nc.index, df_nc.cycl, c='k')
ax[1].set_title('Weekly Cyclicality')
ax[2].plot(df_nc.index, df_nc.R2,c='k')
ax[2].set_xlabel('Day Number')
ax[2].set_ylabel('Remainder After Moving Average')
ax[2].set_title('Remainder')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,12)
plt.show()

In [ ]:
fig,ax = plt.subplots(4,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
#ax[0].set_xlabel('Day Number')
ax[0].set_ylabel('Number of New Cases')
#ax[0].legend()
ax[0].set_title('New Covid Cases')
ax[1].plot(df_nc.index,df_nc.mv_avg, c='k', label='Moving Avg.') # linestyle='--',
ax[1].set_ylabel('Average Number of New Cases')
ax[1].set_title('Moving Average')
ax[2].plot(df_nc.index, df_nc.cycl, c='k')
ax[2].set_ylabel('Average New Cases per Day of Week')
ax[2].set_title('Weekly Cyclicality')
ax[3].plot(df_nc.index, df_nc.R2,c='k')
ax[3].set_xlabel('Day Number')
ax[3].set_ylabel('Remainder After Moving Average')
ax[3].set_title('Remainder')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,16)
plt.show()

# Accuracy

Sum of Squared Errors of Trend + Cyclicality versus Sum of Square Errors around mean new cases.

In [ ]:
import numpy as np
sse_model = (df_nc.R2**2).sum()
sse_data = ((df_nc.positiveIncrease - df_nc.positiveIncrease.mean())**2).sum()
print('Variation explained: %f' % ((sse_data-sse_model)/sse_data,))

# Tickmarks

Integer index values for the month are less than descriptive.  At a minimum, the year should be represented, if not the month.  Here is one way to create better tickmark labels.

In [ ]:
df_nc['date'].iloc[0]

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May' , 'Jun', 'Jul' , 'Aug' , 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
df_nc['date'].iloc[0][0:4] + '-' + months[int(df_nc['date'].iloc[0][5:7]) - 1]

In [ ]:
df_nc.shape[0]

In [ ]:
qtrly_days = [90, 91, 92, 92]
qtrly_days = [qtrly_days[i%4] for i in range(4 * int(df_nc.shape[0]/365))]
qtrly_days

In [ ]:
tickmarks = [sum(qtrly_days[:i]) for i in range(len(qtrly_days) + 1)]
tickmarks

In [ ]:
tickmark_labels = [df_nc['date'].iloc[i][0:4] + '-' + months[int(df_nc['date'].iloc[i][5:7]) - 1] for i in tickmarks]
tickmark_labels

# Graphs with Informative Tickmarks

In [ ]:
fig,ax = plt.subplots(3,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
ax[0].plot(df_nc.index,df_nc.mv_avg, c='r', linestyle='--', label='Moving Avg.')
ax[0].set_xlabel('Day Number')
ax[0].set_ylabel('Number of New Cases')
ax[0].legend()
ax[0].set_title('Moving Average')
'''      Tickmark code      '''
ax[0].xaxis.set_ticks(tickmarks)
ax[0].xaxis.set_ticklabels(tickmark_labels)
'''                         '''
ax[1].plot(df_nc.index, df_nc.cycl, c='k')
ax[1].set_title('Weekly Cyclicality')
ax[2].plot(df_nc.index, df_nc.R2,c='k')
ax[2].set_xlabel('Day Number')
ax[2].set_ylabel('Remainder After Moving Average')
ax[2].set_title('Remainder')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,12)
plt.show()

In [ ]:
fig,ax = plt.subplots(4,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
#ax[0].set_xlabel('Day Number')
ax[0].set_ylabel('Number of New Cases')
#ax[0].legend()
ax[0].set_title('New Covid Cases')
'''      Tickmark code      '''
ax[0].xaxis.set_ticks(tickmarks)
ax[0].xaxis.set_ticklabels(tickmark_labels)
'''                         '''
ax[1].plot(df_nc.index,df_nc.mv_avg, c='k', label='Moving Avg.') # linestyle='--',
ax[1].set_ylabel('Average Number of New Cases')
ax[1].set_title('Moving Average')
ax[2].plot(df_nc.index, df_nc.cycl, c='k')
ax[2].set_ylabel('Average New Cases per Day of Week')
ax[2].set_title('Weekly Cyclicality')
ax[3].plot(df_nc.index, df_nc.R2,c='k')
ax[3].set_xlabel('Day Number')
ax[3].set_ylabel('Remainder After Moving Average')
ax[3].set_title('Remainder')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,16)
fig.savefig('tsa_1.jpg', dpi=600)
plt.show()

In [ ]:
fig,ax = plt.subplots(3,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
ax[0].plot(df_nc.index,df_nc.mv_avg, c='r', linestyle='--', label='Moving Avg.')
ax[0].set_ylabel('Number of New Cases')
ax[0].xaxis.set_ticks(tickmarks)
ax[0].xaxis.set_ticklabels(tickmark_labels)
ax[0].legend()
ax[0].set_title('Moving Average')
ax[1].plot(df_nc.index, df_nc.cycl, c='k')
ax[1].set_title('Weekly Cyclicality')
ax[2].plot(df_nc.index, df_nc.R2,c='k')
ax[2].set_xlabel('Date')
ax[2].set_ylabel('Remainder After Moving Average')
ax[2].set_title('Remainder')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,12)
fig.savefig('tsa_2.jpg', dpi=600)
plt.show()

In [ ]:
fig,ax = plt.subplots(3,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
ax[0].plot(df_nc.index,df_nc.mv_avg, c='r', linestyle='--', label='Moving Avg.')
ax[0].set_ylabel('Number of New Cases')
ax[0].xaxis.set_ticks(tickmarks)
ax[0].xaxis.set_ticklabels(tickmark_labels)
ax[0].legend()
ax[0].set_title('Trend')
ax[1].plot(df_nc.index, df_nc.cycl, c='k')
ax[1].set_title('Weekly Cyclicality')
ax[2].plot(df_nc.index, df_nc.R2,c='k')
ax[2].set_xlabel('Date')
ax[2].set_ylabel('Remainder After Moving Average')
ax[2].set_title('Remainder')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,12)
fig.savefig('tsa_3.jpg', dpi=600)
plt.show()

In [ ]:
fig,ax = plt.subplots(4,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
#ax[0].set_xlabel('Day Number')
ax[0].set_ylabel('Number of New Cases')
#ax[0].legend()
ax[0].set_title('New Covid Cases')
'''      Tickmark code      '''
ax[0].xaxis.set_ticks(tickmarks)
ax[0].xaxis.set_ticklabels(tickmark_labels)
'''                         '''
ax[1].plot(df_nc.index,df_nc.mv_avg, c='k', label='Moving Avg.') # linestyle='--',
ax[1].set_ylabel('Average Number of New Cases')
ax[1].set_title('Trend')
ax[2].plot(df_nc.index, df_nc.cycl, c='k')
ax[2].set_ylabel('Average New Cases per Day of Week')
ax[2].set_title('Weekly Cyclicality')
ax[3].plot(df_nc.index, df_nc.R2,c='k')
ax[3].set_xlabel('Date')
ax[3].set_ylabel('Remainder After Moving Average')
ax[3].set_title('Remainder')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,16)
fig.savefig('tsa_4.jpg', dpi=600)
plt.show()

In [ ]:
fig,ax = plt.subplots(2,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
ax[0].plot(df_nc.index,df_nc.mv_avg, c='r', linestyle='--', label='Moving Avg.')
ax[0].set_ylabel('Number of New Cases')
ax[0].xaxis.set_ticks(tickmarks)
ax[0].xaxis.set_ticklabels(tickmark_labels)
ax[0].legend()
ax[0].set_title('Trend')
ax[1].plot(df_nc.index, df_nc.cycl, c='k')
ax[1].set_title('Weekly Cyclicality')
ax[1].set_xlabel('Date')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,12)
fig.savefig('tsa_5.jpg', dpi=600)
plt.show()

In [ ]:
fig,ax = plt.subplots(3,1, sharex=True)
ax[0].plot(df_nc.index, df_nc.positiveIncrease, c='k', label='Data')
#ax[0].set_xlabel('Day Number')
ax[0].set_ylabel('Number of New Cases')
#ax[0].legend()
ax[0].set_title('New Covid Cases')
'''      Tickmark code      '''
ax[0].xaxis.set_ticks(tickmarks)
ax[0].xaxis.set_ticklabels(tickmark_labels)
'''                         '''
ax[1].plot(df_nc.index,df_nc.mv_avg, c='k', label='Moving Avg.') # linestyle='--',
ax[1].set_ylabel('Average Number of New Cases')
ax[1].set_title('Trend')
ax[2].plot(df_nc.index, df_nc.cycl, c='k')
ax[2].set_ylabel('Average New Cases per Day of Week')
ax[2].set_title('Weekly Cyclicality')
ax[2].set_xlabel('Date')
for i in range(len(ax)):
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
fig.set_size_inches(8,12)
fig.savefig('tsa_6.jpg', dpi=600)
plt.show()

# Reporting Results in a Presentation

![CovidGraphs](CovidGraphs/Slide1.jpg)
![CovidGraphs](CovidGraphs/Slide2.jpg)
![CovidGraphs](CovidGraphs/Slide3.jpg)
![CovidGraphs](CovidGraphs/Slide4.jpg)
![CovidGraphs](CovidGraphs/Slide5.jpg)
![CovidGraphs](CovidGraphs/Slide6.jpg)
![CovidGraphs](CovidGraphs/Slide7.jpg)